# Exercise — Build a Computational Workflow From Business Rules

## Scenario

A client uses a simple rule-based simulation to decide daily staffing. You must implement the logic as a reusable, testable function block. You are given the rules:
1. Expected workload for the day = `forecast_demand` * `demand_variability_factor`
2. Required staff = `ceil(workload/staff_capacity)`
3. If required staff exceeds `max_staff`, cap it at `max_staff`
4. If expected workload < `emergency_threshold`:
    - Set staff = `minimum_staff`
5. Output must contain intermediate values for monitoring.

### Input example

```python
config = {
    "forecast_demand": 120,
    "demand_variability_factor": 1.15,
    "staff_capacity": 30,
    "minimum_staff": 2,
    "max_staff": 10,
    "emergency_threshold": 20,
}
```

### Task
Implement:
```python
def compute_staffing(config: dict) -> dict:
    ...
```

### Requirements
Return:
```python
{
    "expected_workload": float,
    "required_staff_raw" int,
    "final_staff": int
}
```

### Constraints
- No hard-coded values
- Modular, readable logic
- Use math.ceil()

### Evaluation Criteria
- Clean workflow
- Business rules $\rightarrow$ computational logic translation
- Good variable naming
- Testability

## Thinking through the problem

The problem: turn a set of business rules into a small deterministic computation. No optimization solver, just clean logic.

### 1. Inputs (`config` dict)
We get a `config: Dict[str, Any]` with these keys:
- `forecast_demand: float`
- `demand_variability_factor: float`
- `staff_capacity: float`
- `minimum_staff: int`
- `max_staff: int`
- `emergency_threshold: float`

We can think:
- `forecast_demand`: base demand, e.g. expected number of tasks / calls / orders.
- `demand_variability_factor`: multiplier for uncertainty / peaks.
- `staff_capacity`: how much workload one staff member can handle.
- `minimum_staff`: hard lower bound (e.g. must have at least X people).
- `max_staff` hard upper bound (e.g. can’t schedule more than Y people). 
- `emergency_threshold` if workload is tiny, we go to a minimal “skeleton crew”.


### 2. Rule 1 - expected workload

```python
expected_workload = forecast_demand * demand_variability_factor
```
This is the adjusted demand considering variability.



### 3. Rule 2 – required_staff_raw

> `required_staff_raw = ceil(expected_workload / staff_capacity)`
We want enough people so that:
$$
\text{required\_staff} \geq \frac{\text{expected\_workload}}{\text{staff\_capacity}}
$$
So we use ceiling to round up:
```python
required_staff_raw = math.ceil(expected_workload / staff_capacity)
```

Why "raw"? Because we later cap it by `max_staff` and possibly override it.

Note: `staff_capacity` should not be 0; the problem doesn't mention this edge case, so we assume valid config.


### 4. Rule 3 – cap with max_staff

> if `required_staff_raw > max_staff` $\rightarrow$ cap to `max_staff`
So:
```python
required_staff_capped = min(required_staff_raw, max_staff)
```
This ensures we never exceed `max_staff`.

### 5. Rule 4 & 5 - emergency threshold vs `final_staff`

Rules:
4. If `expected_workload < emergency_threshold` $\rightarrow$ `final_staff = minimum_staff`
5. Else $\rightarrow$ `final_staff = capped required_staff`
So we compare the workload, not the staff number.

```python
if expected_workload < emergency_threshold:
    final_staff = minimum_staff
else:
    final_staff = required_staff_capped
```
<!-- Note: the problem doesn’t explicitly say to enforce minimum_staff otherwise, but in practice:

In the “else” case, `required_staff_capped` might already be $\geq$ `minimum_staff` if the demand is higher, so you’re fine.

If we want to be extra safe, we could do `final_staff = max(minimum_staff, required_staff_capped)`, but that’s not exactly what’s written. So let'sstick literally to the rules of the problem. -->


### 6. Return structure

The problem asks for:
```python
{
    "expected_workload": float,
    "required_staff_raw" int,
    "final_staff": int
}
```
Important:
- `expected_workload` is the value before any caps or thresholds.
- `required_staff_raw` is the `ceil(...)` result before `max_staff` cap.
- `final_staff` reflects the actual decision, after:
    - capping by `max_staff`, and
    - possible override by `minimum_staff` due to `emergency_threshold`.


### 7. Quick mental example (using the demo config)
```python
config = {
    "forecast_demand": 120,
    "demand_variability_factor": 1.15,
    "staff_capacity": 30,
    "minimum_staff": 2,
    "max_staff": 10,
    "emergency_threshold": 20,
}
```
Let's compute in our heads:
1. `expected_workload = 120 * 1.15 = 138.0`
2. `required_staff_raw = ceil(138.0 / 30) = 5`
3. Cap with `max_staff=10`: `required_staff_capped = min(5, 10) = 5`
4. Emergency rule: compare workload to `emergency_threshold=20`
    - `expected_workload=138.0` is greater than `20`, so we don't override.
    - So `final_staff = 5`

So, our function should return:
```python
{
    "expected_workload": 138.0,
    "required_staff_raw": 5,
    "final_staff": 5
}
If we run `forecast_demand` small enough (e.g. 5, factor of 1.0, capacity 30 $\rightarrow$ expected workload 5 < 20), then:
- It will set `final_staff = minimum_staff` regardless of `required_staff_raw`.

## Code implementation

In [1]:
import math 
from typing import Dict, Any

def compute_staffing(config: Dict[str, Any]) -> Dict[str, Any]:
    """
    Compute staffing levels based on simple business rules.
    
    This function demonstrates how to transform business rules into a clean,
    testable computational workflow. Key principles:
    1. Config-driven: no hard-coded values, making it reusable across scenarios
    2. Step-by-step logic: each business rule maps to a clear computational step
    3. Transparent: returns intermediate values for monitoring and debugging
    4. Defensive: validates inputs to catch errors early

    Config keys:
        - forecast_demand: float - Base expected demand (e.g., tasks, calls, orders)
        - demand_variability_factor: float - Multiplier to account for uncertainty/peaks
        - staff_capacity: float - How much workload one staff member can handle
        - minimum_staff: int - Hard lower bound (must have at least this many)
        - max_staff: int - Hard upper bound (cannot schedule more than this)
        - emergency_threshold: float - If workload below this, use minimal skeleton crew

    Returns:
        Dictionary containing:
          - expected_workload: Adjusted demand after variability factor
          - required_staff_raw: Calculated staff before constraints (for transparency)
          - final_staff: Final staffing decision after all rules applied
    """
    # STEP 1: Extract and type-convert values from config dictionary
    # Why extract first? Makes code readable, enables validation, and provides
    # clear variable names that match the business domain terminology
    forecast_demand = float(config["forecast_demand"])
    demand_variability_factor = float(config["demand_variability_factor"])
    staff_capacity = float(config["staff_capacity"])
    minimum_staff = int(config["minimum_staff"])
    max_staff = int(config["max_staff"])
    emergency_threshold = float(config["emergency_threshold"])

    # STEP 2: Compute expected workload (Business Rule 1)
    # This adjusts the base forecast by a variability factor to account for
    # uncertainty, peak periods, or safety margins
    # Example: forecast_demand=100, factor=1.15 → expected_workload=115.0
    expected_workload = forecast_demand * demand_variability_factor

    # STEP 3: Calculate raw staffing requirement (Business Rule 2)
    # We need enough staff to handle the expected workload
    # math.ceil ensures we always round UP (e.g., 7.1 workload needs 8 staff)
    # This is critical: understaffing can cause service degradation
    # 
    # Input validation: prevent division by zero or negative capacity
    if staff_capacity <= 0:
        raise ValueError("staff_capacity must be positive.")
    
    # Calculate: required_staff = ceil(workload / capacity_per_staff)
    # Example: workload=138, capacity=30 → 138/30 = 4.6 → ceil(4.6) = 5
    required_staff_raw = int(math.ceil(expected_workload / staff_capacity))

    # STEP 4: Apply maximum staff constraint (Business Rule 3)
    # Cap the required staff at the maximum allowed (budget/resource constraint)
    # This ensures we never exceed max_staff, even if workload is very high
    # Example: required_staff_raw=12, max_staff=10 → required_staff_capped=10
    required_staff_capped = min(required_staff_raw, max_staff)

    # STEP 5: Apply emergency threshold rule (Business Rule 4)
    # If workload is below a critical threshold, switch to minimal staffing
    # This handles low-demand scenarios efficiently (skeleton crew mode)
    # Note: We compare workload (not staff count) to the threshold
    if expected_workload < emergency_threshold:
        # Low-demand mode: use minimum staffing regardless of calculation
        final_staff = minimum_staff
    else:
        # Normal mode: use the capped required staff
        final_staff = required_staff_capped

    # STEP 6: Return structured result with all intermediate values
    # Why return intermediates? Enables:
    #   - Monitoring: see where adjustments were made (caps, overrides)
    #   - Debugging: trace how final value was computed
    #   - UI display: show users the reasoning behind the decision
    #   - Auditing: track when constraints were applied
    return {
        "expected_workload": float(expected_workload),  # After variability adjustment
        "required_staff_raw": required_staff_raw,  # Before max_staff cap
        "final_staff": final_staff,  # Final decision after all rules
    }

# ============================================================
# Test algorithm
# ============================================================
# if __name__ == "__main__":

config_demo = {
    "forecast_demand": 120,
    "demand_variability_factor": 1.15,
    "staff_capacity": 30,
    "minimum_staff": 2,
    "max_staff": 10,
    "emergency_threshold": 20,
}

result4 = compute_staffing(config_demo)
print("Exercise 4 result:", result4)

Exercise 4 result: {'expected_workload': 138.0, 'required_staff_raw': 5, 'final_staff': 5}
